### 线性回归

线性回归是机器学习中的基本算法之一。 线性回归在输入特征 (X) 和输出标签 (y) 之间建立线性关系。

在线性回归中，每个输出标签都表示为使用权重和偏差的输入特征的线性函数。 这些权重和偏差是随机初始化的模型参数，然后通过数据集的每个训练/学习周期进行更新。 在经过一次训练数据迭代后训练模型和更新参数被称为一个时期。 所以现在我们应该训练模型几个时期，以便权重和偏差可以学习输入特征和输出标签之间的线性关系。

例子我们将根据特定地点的平均温度、年降雨量和湿度，创建一个由芒果和橙子的作物产量组成的假设数据模型。

Mangoes = w11 * temp + w12 * rainfall + w13 * humidity + b1 

Oranges = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [1]:
import torch
import numpy as np

In [2]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [4]:
print(inputs)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])


In [5]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


In [6]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(inputs, targets)

In [7]:
dataset[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [8]:
from torch.utils.data import DataLoader

batch_size = 3
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
# A Batch Sample
for inp,target in train_loader:
    print(inp)
    print(target)
    break


tensor([[ 91.,  88.,  64.],
        [ 68.,  96.,  71.],
        [ 87., 135.,  57.]])
tensor([[ 81., 101.],
        [104., 118.],
        [118., 134.]])


从头开始实现一个线性回归模型。 我们应该找到上面等式中指定的最佳权重和偏差，以便它定义输入和输出之间的理想线性关系

In [10]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.4496e+00, -3.3164e-01, -1.4695e-03],
        [-1.9847e+00, -7.8420e-01,  4.0560e-03]], requires_grad=True)
tensor([-0.6681, -0.2995], requires_grad=True)


In [11]:
def model(X):
    return X @ w.t() + b

In [12]:
for x,y in train_loader:
    preds = model(x)
    print("Prediction is :n",preds)
    print("nActual targets is :n",y)
    break

Prediction is :n tensor([[  83.1969, -196.7641],
        [ 102.2937, -248.8724],
        [  65.9602, -210.2570]], grad_fn=<AddBackward0>)
nActual targets is :n tensor([[ 57.,  69.],
        [ 80., 102.],
        [104., 118.]])


In [13]:
def mse_loss(predictions, targets):
    difference = predictions - targets
    return torch.sum(difference * difference)/ difference.numel()

In [14]:
for x,y in train_loader:
    preds = model(x)
    print("Prediction is :n",preds)
    print("nActual targets is :n",y)
    print("nLoss is: ",mse_loss(preds, y))
    break

Prediction is :n tensor([[  80.5884, -278.6077],
        [  67.4112, -212.2458],
        [ 132.8723, -236.3132]], grad_fn=<AddBackward0>)
nActual targets is :n tensor([[118., 134.],
        [103., 119.],
        [ 22.,  37.]])
nLoss is:  tensor(61604.6406, grad_fn=<DivBackward0>)


In [17]:
epochs = 50
for i in range(epochs):
    # Iterate through training dataloader
    for x,y in train_loader:
        # Generate Prediction
        preds = model(x)
        # Get the loss and perform backpropagation
        loss = mse_loss(preds, y)
        loss.backward()
        # Let's update the weights
        with torch.no_grad():
            w -= w.grad *1e-6
            b -= b.grad * 1e-6
            # Set the gradients to zero
            w.grad.zero_()
            b.grad.zero_()
        print(f"Epoch {i}/{epochs}: Loss: {loss}")

Epoch 0/50: Loss: 61412.234375
Epoch 0/50: Loss: 49319.40625
Epoch 0/50: Loss: 52399.94921875
Epoch 0/50: Loss: 47296.625
Epoch 0/50: Loss: 53077.19140625
Epoch 1/50: Loss: 42723.45703125
Epoch 1/50: Loss: 38543.38671875
Epoch 1/50: Loss: 59146.80078125
Epoch 1/50: Loss: 33769.9765625
Epoch 1/50: Loss: 47197.15625
Epoch 2/50: Loss: 36682.85546875
Epoch 2/50: Loss: 46485.5625
Epoch 2/50: Loss: 38457.91015625
Epoch 2/50: Loss: 32818.33984375
Epoch 2/50: Loss: 31486.59375
Epoch 3/50: Loss: 33356.796875
Epoch 3/50: Loss: 26187.927734375
Epoch 3/50: Loss: 31339.9609375
Epoch 3/50: Loss: 34430.23046875
Epoch 3/50: Loss: 31001.322265625
Epoch 4/50: Loss: 29939.220703125
Epoch 4/50: Loss: 30117.255859375
Epoch 4/50: Loss: 25905.005859375
Epoch 4/50: Loss: 22205.5234375
Epoch 4/50: Loss: 23500.169921875
Epoch 5/50: Loss: 20402.265625
Epoch 5/50: Loss: 20036.771484375
Epoch 5/50: Loss: 25021.177734375
Epoch 5/50: Loss: 27657.431640625
Epoch 5/50: Loss: 18041.07421875
Epoch 6/50: Loss: 17318.9863

In [18]:
for x,y in train_loader:
    preds = model(x)
    print("Prediction is :n",preds)
    print("nActual targets is :n",y)
    break

Prediction is :n tensor([[ 89.7322,  96.8692],
        [ 61.7146, 120.6202],
        [ 90.8762,  95.4166]], grad_fn=<AddBackward0>)
nActual targets is :n tensor([[ 80., 102.],
        [104., 118.],
        [ 82., 100.]])
